## Analytical Results based on Renewal Theory
**Stochastic Kinetics of mRNA Molecules in a General Transcription Model**

*Yuntao Lu and Yunxin Zhang*

School of Mathematical Sciences, Fudan University, Shanghai 200433, China

Email: `yuntaolu22@m.fudan.edu.cn` and `xyz@fudan.edu.cn`

This script is written to time computations using Eq.(14), generating date used to plot Figure 4. 

In [1]:
import numpy as np
from scipy.integrate import quad
import scipy.linalg as linalg
import time

In [5]:
def renewal_time(D0, D1):
    """
    Parameters:
    -----------
    D0 : np.ndarray
    D1 : np.ndarray

    Returns:
    --------
    float : Execution time in seconds.
    """
    
    # --- Initialization ---
    
    N = D0.shape[0]  # Number of states (dimension of the square matrix D0)
    
    # Initialize uniform probability vector: k = [1/N, 1/N, ..., 1/N] (1×N row vector)
    k = np.full((1, N), 1.0 / N)
    
    # Column vector of ones (N×1), used for summing over states.
    e = np.ones((N, 1))
    
    d = 1

    # --- Define Core Functions ---
    
    def f(t):
        """
        Probability density function at time t: f(t) = k * exp(D0*t) * D1 * e
        
        Parameters:
        -----------
        t : float
            Time point (> 0).
            
        Returns:
        --------
        float : Density value at time t.
        """
        # Compute matrix exponential: exp(D0 * t)
        exp_D0_t = linalg.expm(D0 * t)
        # Apply transformations: k @ exp_D0_t @ D1 @ e → scalar
        result=k @ exp_D0_t @ D1 @ e
        return result

    def Laplace(s):
        """
        Laplace transform of f(t) evaluated at real s.
        
        L(s) = int_0^inf f(t) * exp(-s*t) dt
        
        Parameters:
        -----------
        s : float
            
        Returns:
        --------
        float: Value of the Laplace transform.
        """
        def integrand(t):
            """Integrand for Laplace transform: f(t) * exp(-s*t)"""
            return f(t) * np.exp(-s * t)
        
        # Numerical integration from 0 to infinity
        result, error = quad(integrand, 0, np.inf, epsabs=1.49e-8, epsrel=1.49e-8)
        return result

    def meantime():
        """
        Compute the expected (mean) renewal time: E[T] = int_0^inf t * f(t) dt
        
        Returns:
        --------
        float : Mean time of inter-arrival time.
        """
        def integrand(t):
            """Integrand for mean computation: t * f(t)"""
            return t * f(t)
        
        result, error = quad(integrand, 0, np.inf, epsabs=1.49e-8, epsrel=1.49e-8)
        return result

    # --- Timing Start ---
    start_time = time.perf_counter()

    # --- Compute Mean Time ---
    a = meantime()

    # --- Generate Binomial Coefficient Sequence ---
    # Binomial[0] = 1 (index 0 corresponds to i=1 in loop)
    Binomial = [1]
    
    # First coefficient: B_1 = 1 / (d * a)
    B = 1.0 / (d * a)
    Binomial.append(B)

    # Compute next 199 terms (i from 2 to 200 inclusive → total 200 after initial 1)
    for i in range(2, 201):
        # Recursive update using Laplace transform at (i-1)
        # B_i = (i-1)/i * B_{i-1} * L(i-1)/(1 - L(i-1))
        Lap = Laplace(i - 1)
        denominator = 1 - Lap
        
        # if abs(denominator) < 1e-15:
        #     # Avoid division by zero; likely Lap ≈ 1 indicates convergence
        #     print(f"Warning: Denominator near zero at i={i}, Lap={Lap}.")
        #     break
            
        B = (i - 1) / i * B * (Lap / denominator)
        Binomial.append(B)

    # --- Timing End ---
    end_time = time.perf_counter()
    execution_time = end_time - start_time

    # Output execution time with high precision
    print(f"Execution time of {N}-th order model: {execution_time:.8f} seconds")

    # Optional: Return Binomial coefficients or other results?
    # Currently only returning timing per original code.
    return execution_time

In [3]:
# import parameter matrices prepared for figures in the paper
import Parameters_for_Figures

In [6]:
times=[]
for i in range(1,51):
    time1=renewal_time(Parameters_for_Figures.generate_D0n(i),
                     Parameters_for_Figures.generate_D1n(i)
                     )
    time2=renewal_time(Parameters_for_Figures.generate_D0n(i),
                     Parameters_for_Figures.generate_D1n(i)
                     )
    time3=renewal_time(Parameters_for_Figures.generate_D0n(i),
                     Parameters_for_Figures.generate_D1n(i)
                     )
    times.append(float(np.average([time1,time2,time3])))

Execution time of 1-th order model: 0.34588090 seconds
Execution time of 1-th order model: 0.29285249 seconds
Execution time of 1-th order model: 0.28964429 seconds
Execution time of 2-th order model: 0.60295053 seconds
Execution time of 2-th order model: 0.39534429 seconds
Execution time of 2-th order model: 0.35416448 seconds
Execution time of 3-th order model: 0.37540214 seconds
Execution time of 3-th order model: 0.37389336 seconds
Execution time of 3-th order model: 0.37243941 seconds
Execution time of 4-th order model: 0.39983863 seconds
Execution time of 4-th order model: 0.37549040 seconds
Execution time of 4-th order model: 0.40071145 seconds
Execution time of 5-th order model: 0.40778191 seconds
Execution time of 5-th order model: 0.40419745 seconds
Execution time of 5-th order model: 0.40814406 seconds
Execution time of 6-th order model: 0.41889840 seconds
Execution time of 6-th order model: 0.40966640 seconds
Execution time of 6-th order model: 0.42855195 seconds
Execution 

In [10]:
print(times)

[0.30945922682682675, 0.4508197655280431, 0.37391163657108945, 0.39201349516709644, 0.40670781085888547, 0.41903891662756604, 0.44558533777793247, 0.46292363852262497, 0.475601760049661, 0.48967111110687256, 0.5071065252025923, 0.5271452317635218, 0.5442487001419067, 0.5487626989682516, 0.5602186148365339, 0.5814718504746755, 0.6170949215690295, 0.6005963782469431, 0.5934355035424232, 0.6153270453214645, 0.6361025969187418, 0.688476358850797, 0.6631260514259338, 0.6874157935380936, 0.710400881866614, 0.7143861378232638, 0.718808022638162, 0.7244263341029485, 0.7287646308541298, 0.7406751612822214, 0.7382701759537061, 0.7579755187034607, 0.7932920753955841, 0.8373892679810524, 0.8651008531451225, 0.8809586986899376, 0.8785086994369825, 0.8936128492156664, 0.9053063169121742, 0.9112539514899254, 0.9279759128888448, 0.9435001189510027, 0.9587390497326851, 0.9792389695843061, 0.9865707109371821, 0.9831356604894003, 1.004328986008962, 1.0617395490407944, 1.1356075281898181, 1.15199020753304

In [8]:
# Save the results in a `.npy` file
# np.save('Renewal_times.npy', times)

In [9]:
# load .npy file
loaded_array = np.load('Renewal_times.npy')
loaded_list_from_array = loaded_array.tolist()
print(loaded_list_from_array)

[0.30945922682682675, 0.4508197655280431, 0.37391163657108945, 0.39201349516709644, 0.40670781085888547, 0.41903891662756604, 0.44558533777793247, 0.46292363852262497, 0.475601760049661, 0.48967111110687256, 0.5071065252025923, 0.5271452317635218, 0.5442487001419067, 0.5487626989682516, 0.5602186148365339, 0.5814718504746755, 0.6170949215690295, 0.6005963782469431, 0.5934355035424232, 0.6153270453214645, 0.6361025969187418, 0.688476358850797, 0.6631260514259338, 0.6874157935380936, 0.710400881866614, 0.7143861378232638, 0.718808022638162, 0.7244263341029485, 0.7287646308541298, 0.7406751612822214, 0.7382701759537061, 0.7579755187034607, 0.7932920753955841, 0.8373892679810524, 0.8651008531451225, 0.8809586986899376, 0.8785086994369825, 0.8936128492156664, 0.9053063169121742, 0.9112539514899254, 0.9279759128888448, 0.9435001189510027, 0.9587390497326851, 0.9792389695843061, 0.9865707109371821, 0.9831356604894003, 1.004328986008962, 1.0617395490407944, 1.1356075281898181, 1.15199020753304